In [4]:
!pip install requests
!pip install lxml
!pip install selenium
!pip install unidecode
!pip install beautifulsoup4

     -------------------------------------- 147.9/147.9 kB 4.4 MB/s eta 0:00:00


In [1]:
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from unidecode import unidecode
import requests
import lxml as lxml
import html as html
import time
import pandas as pd
import numpy as np 
import re

In [2]:
# Inicializa el driver de Selenium (asegúrate de tener el driver correspondiente instalado)
driver = webdriver.Chrome()  # Puedes cambiar a otro navegador y/o ruta del driver si lo necesitas
driver.maximize_window()

#Link a la pagina web
link = 'https://diaonline.supermercadosdia.com.ar/'
link_sin_barra = 'https://diaonline.supermercadosdia.com.ar'

#Abre la página web en el navegador controlado por Selenium
driver.get(link)


#Hacemos click en el boton de categorias
button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'diaio-custom-mega-menu-0-x-custom-mega-menu-trigger__button')))
button.click()
#time.sleep(2)

# Obtiene el HTML después de cargar todos los elementos
html = driver.page_source

# Cierra el navegador controlado por Selenium
driver.quit()

# Parsea el HTML con BeautifulSoup
soup_1 = BeautifulSoup(html, 'lxml')

#Buscamos la lista de las categorias 
containers = soup_1.find_all(class_ = "diaio-custom-mega-menu-0-x-custom-mega-menu-desktop__list-item") #Aca filtramos por el elemento UL que contiene las categorias

# Lista para almacenar los elementos filtrados
categorias = []

# Itera sobre los contenedores para filtrar los que contienen un div interno con la clase "diaio-store-5-x-menuItem"
for container in containers:
    c = container.find_all('a', class_='diaio-custom-mega-menu-0-x-category-list-item__container')
    for i in c:
        # Obtiene el texto después de </span>
        categoria = i.find('span', class_='diaio-custom-mega-menu-0-x-category-list-item__icon-container').next_sibling.strip()
        texto_procesado = unidecode(categoria).lower()
        texto_url = texto_procesado.replace(' ','-')
        categorias.append(texto_url)

categorias

['almacen',
 'bebidas',
 'frescos',
 'desayuno',
 'limpieza',
 'perfumeria',
 'congelados',
 'bebes-y-ninos',
 'hogar-y-deco',
 'mascotas',
 'kiosco',
 'frutas-y-verduras',
 'electro-hogar']

In [3]:
#Primer categoria
driver = webdriver.Chrome()  # Puedes cambiar a otro navegador y/o ruta del driver si lo necesitas
driver.maximize_window()
primer_url = link + categorias[0]


# Abre la página web en el navegador controlado por Selenium
driver.get(primer_url)
button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "vtex-search-result-3-x-seeMoreButton")))
time.sleep(3)
driver.execute_script("arguments[0].click();", button)
time.sleep(3)

# Obtiene el HTML después de cargar todos los elementos
html_2 = driver.page_source

# Cierra el navegador controlado por Selenium
driver.quit()

In [4]:
# Parsea el HTML con BeautifulSoup
soup_2 = BeautifulSoup(html_2, 'lxml')
containers = soup_2.find_all('div', class_='vtex-search-result-3-x-filter__container')
containers


#creamos el filtro que nos devuelve las sub-categorias
filtros = []
sub_categorias_sin_procesar = []
sub_categorias = []

for i in range(len(containers)):
    value = containers[i].get_text()
    filtros.append(value)
    test = re.findall('Sub-Categoría', filtros[i])
    if test:
        texto_lista = containers[i].get_text(separator='\n', strip=True).split('\n')
        sub_categorias_sin_procesar.extend(texto_lista[1:-1]) #Exluimos el ultimo registro

#limpiamos el texto para poder generar las variables que necesitamos
for i in sub_categorias_sin_procesar:
    texto_procesado = unidecode(i).lower()
    texto = texto_procesado.replace(' ','-')
    texto_2 = texto.replace(',', '')
    sub_categorias.append(texto_2)
sub_categorias 

['aceites-blend',
 'aceites-de-girasol',
 'aceites-de-oliva',
 'aceites-de-soja',
 'aceitunas-y-encurtidos',
 'acetos-vinagres-y-limon',
 'alfajores',
 'arroz-doble',
 'arroz-integral',
 'arroz-largo',
 'arroz-parboil',
 'arroz-preparado',
 'bizcochuelos',
 'bolsas-saborizadoras',
 'budines-y-magdalenas',
 'caldos',
 'caramelos-y-gomitas',
 'chicles-y-chupetines',
 'chocolates',
 'coberturas',
 'comidas-elaboradas',
 'conservas-de-frutas',
 'conservas-de-pescados',
 'conservas-de-vegetales',
 'especias',
 'facturas-y-medialunas',
 'fideos-largos',
 'fideos-para-guiso',
 'fideos-para-sopa',
 'gelatinas',
 'grisines-y-tostadas',
 'harinas-de-maiz',
 'harinas-de-trigo',
 'ketchup',
 'legumbres-y-semillas',
 'mayonesa',
 'mostaza',
 'otras-especialidades',
 'otros-aderezos',
 'pan-de-hamburguesa-y-pancho',
 'pan-de-molde',
 'pan-dulce',
 'pan-rallado-y-rebozadores',
 'panes',
 'papas-fritas',
 'pastas-rellenas',
 'polvo-para-hornear-y-esencias',
 'postres-para-preparar',
 'premezclas-dulce

In [5]:
#Primer sub-categoria

#volvemos a entrar a la misma categoria
driver = webdriver.Chrome()  # Puedes cambiar a otro navegador y/o ruta del driver si lo necesitas
driver.maximize_window()
primer_url = link + categorias[0]
driver.get(primer_url)

#Ahora generamos la categoria que queremos seleccionar
button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "vtex-search-result-3-x-seeMoreButton")))#Expandimos todas las secciones
button.click()
time.sleep(3)


selector_sub_categoria = f'label[for="category-3-{sub_categorias[18]}"]' #Creamos la categoria que necesitamos para hacer click
checkbox = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector_sub_categoria)))
checkbox.click()
time.sleep(5)
driver.quit()

In [10]:
driver = webdriver.Chrome()  # Puedes cambiar a otro navegador y/o ruta del driver si lo necesitas
driver.maximize_window()
primer_url = link + categorias[0]
driver.get(primer_url)

# Ahora generamos la categoria que queremos seleccionar
button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "vtex-search-result-3-x-seeMoreButton")))  # Expandimos todas las secciones
button.click()
time.sleep(3)

selector_sub_categoria = f'label[for="category-3-{sub_categorias[18]}"]'  # Creamos la categoria que necesitamos para hacer click
checkbox = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector_sub_categoria)))
checkbox.click()
time.sleep(5)

# Esta parte va a scrollear hasta abajo y actualizar los objetos que componen la lista a obtener
prev_elements_count = len(driver.find_elements(By.CLASS_NAME, 'vtex-search-result-3-x-galleryItem'))

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    curr_elements_count = len(driver.find_elements(By.CLASS_NAME, 'vtex-search-result-3-x-galleryItem'))

    if curr_elements_count == prev_elements_count:
        break

    # Actualizar el contador de elementos
    prev_elements_count = curr_elements_count

print(prev_elements_count)
# Obtiene el HTML después de cargar todos los elementos
html_3 = driver.page_source
time.sleep(2)

# Cierra el navegador controlado por Selenium
driver.quit()

soup_3 = BeautifulSoup(html_3, 'lxml')
productos_categoria = soup_3.find_all('div', class_='vtex-search-result-3-x-galleryItem')
print(len(productos_categoria))

total_items = []

for litem_listado in range(len(productos_categoria)):
    try:
        # Extraemos link de la imagen del producto
        imagen = productos_categoria[litem_listado].find('img')
        link_imagen = imagen['src'] if imagen else None

        # Extraemos link a la web del producto en el supermercado
        web_producto = productos_categoria[litem_listado].find('a')
        url_producto = link_sin_barra + web_producto['href'] if web_producto else None

        # Datos del producto
        informacion_producto_wrapped = productos_categoria[litem_listado].find('div', 'vtex-flex-layout-0-x-flexCol')
        informacion_producto = informacion_producto_wrapped.find_all('div', 'vtex-flex-layout-0-x-flexColChild') if informacion_producto_wrapped else []

        # Creamos las listas soporte
        datos_productos_sin_procesar = []
        datos_productos_procesados = []
        datos_productos_procesados.append(categorias[0])  # Agregamos la categoria a la que pertenece el producto
        datos_productos_procesados.append(sub_categorias[18])  # Agregamos la categoria a la que pertenece el producto

        # Con este bucle for extraemos la informacion sin procesar
        for info in informacion_producto:
            valor_lista = info.get_text(strip=True).replace('\xa0', ' ')
            datos_productos_sin_procesar.append(valor_lista)

        # Generamos la expresion regular que extrae el precio correcto
        # y conformamos la lista con los datos del producto
        for i in range(0, 4):
            datos_productos_procesados.append(datos_productos_sin_procesar[i] if i < len(datos_productos_sin_procesar) else None)

        for i in range(len(datos_productos_sin_procesar)):
            patron_regex_precios = r"(\$\d{1,3}(?:\.\d{3})*,\d{2})"
            resultado_regex = re.findall(patron_regex_precios, datos_productos_sin_procesar[i])
            if resultado_regex:
                datos_productos_procesados.extend(resultado_regex)

        if len(datos_productos_procesados) == 7:
            datos_productos_procesados.append(None)
            datos_productos_procesados.append(datos_productos_sin_procesar[-1])
        else:
            datos_productos_procesados.append(datos_productos_sin_procesar[-1])

        datos_productos_procesados.append(url_producto)  # Agregamos el link al producto
        datos_productos_procesados.append(link_imagen)  # Agregamos el link a la imagen

        total_items.append(datos_productos_procesados)
    except Exception as e:
        print(f"Error procesando el producto {litem_listado}: {e}")

nombre_dataframe = f'data_producto_{categorias[0]}_{sub_categorias[18]}'
lista_columnas = ('categoria', 'producto', 'banner_producto', 'marca', 'descripcion_producto',
                  'promocion', 'precio', 'precio_sin_descuento', 'precio_por_kilo',
                  'url_producto', 'url_imagen')

globals()[nombre_dataframe] = pd.DataFrame(total_items, columns=lista_columnas)
ruta_csv = f'C:/Users/phoen/OneDrive/Escritorio/Lauta/Data Science/Projects/scrapper_dia/Productos obtenidos/{nombre_dataframe}.csv'
globals()[nombre_dataframe].to_csv(ruta_csv, index=False)

87
87
